In [ ]:
from typing import Any
import numpy as np
import random

class RandomWalk:
    def __init__(self) -> None:
        self.state = 9
        pass
    
    def action_space(self):
        return 2
    
    def state_space(self):
        return 19
    
    def reset(self):
        self.state = 9
        return self.state
    
    def step_(self, action):
        new_state = self.state + (-1 if action == 0 else 1)
        reward = 1 if new_state == 19 else 0
        terminated = new_state == -1 or new_state == 19
        self.state = 9 if terminated else new_state
        return terminated, reward, new_state
    
    def step(self):
        return self.step_(random.choice(range(2)))
    